# Web Scrapping

## Analysis of Cost of Living Across Global Cities

### Import Libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

In [3]:
url = "https://www.numbeo.com/cost-of-living/rankings.jsp"

### Request and Fetch the Webpage

In [5]:
requests.get("https://www.numbeo.com/cost-of-living/rankings.jsp")

<Response [200]>

In [6]:
url= "https://www.numbeo.com/cost-of-living/rankings.jsp"
headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=headers)
print("Status Code:",response.status_code)

Status Code: 200


In [7]:
response.text[0:500]

'<!DOCTYPE html>\n<html lang="en-US">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n\n<meta name="description" content="Rankings: Cost of Living Index,  Restaurant Prices Index, Transportation Price Index, Grocery Price Index, Local Purchasing Power Index, ... " />\n            <script defer src="https://www.numbeo.com/common/numbeo_standard.js"></script>\n<link rel="preconnect" href="https://fonts.googleapis.com">\n<link rel="preconnect" href="https://fonts.gstatic.com" c'

### Pass the fetched webpage response to Beautiful Soup

In [9]:
# give the webpage to Beautiful Soup using parsers: "html.parser" or "lxml"
soup = BeautifulSoup(response.text,"html.parser")
soup

<!DOCTYPE html>

<html lang="en-US">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Rankings: Cost of Living Index,  Restaurant Prices Index, Transportation Price Index, Grocery Price Index, Local Purchasing Power Index, ... " name="description">
<script defer="" src="https://www.numbeo.com/common/numbeo_standard.js"></script>
<link href="https://fonts.googleapis.com" rel="preconnect"/>
<link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/>
<link href="https://fonts.googleapis.com/css2?family=Lato:ital,wght@0,100;0,300;0,400;0,700;0,900;1,100;1,300;1,400;1,700;1,900&amp;display=swap" rel="stylesheet"/>
<link href="https://cdnjs.cloudflare.com/ajax/libs/jqueryui/1.10.3/themes/redmond/jquery-ui.min.css" rel="Stylesheet">
<link href="https://www.numbeo.com/favicon.ico" rel="shortcut icon" type="image/x-icon">
<script defer="" src="https://cdnjs.cloudflare.com/ajax/libs/jquery/1.10.2/jquery.min.js"></script>
<script defer="" src

In [10]:
table = soup.find("table", id="t2") 
table

<table class="stripe row-border order-column compact" id="t2">
<thead>
<tr>
<th><div style="font-size: 80%; vertical-align: middle;">Rank</div></th>
<th><div style="font-size: 95%;">City</div></th>
<th><div style="font-size: 90%;">Cost of Living Index</div></th>
<th><div style="font-size: 90%;">Rent Index</div></th>
<th><div style="font-size: 90%;">Cost of Living Plus Rent Index</div></th>
<th><div style="font-size: 90%;">Groceries Index</div></th>
<th><div style="font-size: 90%;">Restaurant Price Index</div></th>
<th><div style="font-size: 90%;">Local Purchasing Power Index</div></th>
</tr>
</thead>
<tbody>
<tr style="width: 100%">
<td></td>
<td class="cityOrCountryInIndicesTable">Zurich, Switzerland</td>
<td style="text-align: right">112.5</td>
<td style="text-align: right">68.1</td>
<td style="text-align: right">93.2</td>
<td style="text-align: right">113.9</td>
<td style="text-align: right">110.4</td>
<td style="text-align: right">167.3</td>
</tr>
<tr style="width: 100%">
<td></td>

### Extract Column Headers

In [11]:
headers_list = table.find_all("th")          
columns = [h.text.strip() for h in headers_list]
columns

['Rank',
 'City',
 'Cost of Living Index',
 'Rent Index',
 'Cost of Living Plus Rent Index',
 'Groceries Index',
 'Restaurant Price Index',
 'Local Purchasing Power Index']

### Extract Data from a Single Page 

In [12]:
tbody = table.find("tbody")                
rows = tbody.find_all("tr")     

In [13]:
current_data = []

for row in rows:
    cols = row.find_all("td")                   
    row_data = [c.text.strip() for c in cols]
    row_data.append("2025")                
    current_data.append(row_data)


### Generate Pagination URLs for All Years

In [14]:
years = list(range(2015, 2024))
all_years_data = []

for year in years:
    
    page_url = f"https://www.numbeo.com/cost-of-living/rankings.jsp?title={year}"
    response = requests.get(page_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    
    table = soup.find("table", id="t2")         
    if table is None:
        continue
    
    tbody = table.find("tbody")                
    rows = tbody.find_all("tr")                 
    
    for row in rows:
        cols = row.find_all("td")              
        row_data = [c.text.strip() for c in cols]
        row_data.append(year)
        all_years_data.append(row_data)
    
    time.sleep(1)

In [15]:
final_columns = columns + ["Year"]
combined_data = current_data + all_years_data
combined_data

[['',
  'Zurich, Switzerland',
  '112.5',
  '68.1',
  '93.2',
  '113.9',
  '110.4',
  '167.3',
  '2025'],
 ['',
  'Geneva, Switzerland',
  '111.4',
  '63.7',
  '90.6',
  '113.8',
  '113.4',
  '171.0',
  '2025'],
 ['',
  'Basel, Switzerland',
  '110.7',
  '49.1',
  '83.9',
  '116.2',
  '108.4',
  '194.8',
  '2025'],
 ['',
  'Lausanne, Switzerland',
  '110.5',
  '48.3',
  '83.4',
  '110.7',
  '111.3',
  '180.4',
  '2025'],
 ['',
  'Lugano, Switzerland',
  '108.4',
  '41.2',
  '79.1',
  '111.0',
  '111.4',
  '159.7',
  '2025'],
 ['',
  'Bern, Switzerland',
  '103.4',
  '43.2',
  '77.2',
  '106.0',
  '91.1',
  '186.8',
  '2025'],
 ['',
  'New York, NY, United States',
  '100.0',
  '100.0',
  '100.0',
  '100.0',
  '100.0',
  '100.0',
  '2025'],
 ['',
  'Reykjavik, Iceland',
  '96.2',
  '50.5',
  '76.3',
  '102.1',
  '106.5',
  '113.1',
  '2025'],
 ['',
  'Honolulu, HI, United States',
  '94.4',
  '57.9',
  '78.5',
  '120.2',
  '83.7',
  '108.7',
  '2025'],
 ['',
  'San Francisco, CA, United

## Create a DataFrame

In [16]:
df = pd.DataFrame(combined_data, columns=final_columns)
df

,Rank,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,Year
0,,"Zurich, Switzerland",112.5,68.1,93.2,113.9,110.4,167.3,2025
1,,"Geneva, Switzerland",111.4,63.7,90.6,113.8,113.4,171.0,2025
2,,"Basel, Switzerland",110.7,49.1,83.9,116.2,108.4,194.8,2025
3,,"Lausanne, Switzerland",110.5,48.3,83.4,110.7,111.3,180.4,2025
4,,"Lugano, Switzerland",108.4,41.2,79.1,111.0,111.4,159.7,2025
...,...,...,...,...,...,...,...,...,...
4960,,"Bucaramanga, Colombia",19.6,4.6,12.4,16.1,13.6,31.3,2023
4961,,"Rawalpindi, Pakistan",16.4,2.9,9.9,14.7,12.5,27.9,2023
4962,,"Faisalabad, Pakistan",16.1,2.8,9.7,12.0,10.5,32.5,2023
4963,,"Multan, Pakistan",15.6,2.5,9.3,15.1,11.0,21.9,2023


In [17]:
df.shape

(4965, 9)

In [46]:
df.to_csv("numbeo_cost_of_living_all_years.csv", index=False)